# Presentation

#### Goal: Predict the number of nights granted to a request (not to an individual)


#### Caracteristics of the model:
Accuracy:
- if used as a clearing tool removing overburden upfront, and thus only to get rid of obvious cases, the accuracy of such a tool could be its most important caracteristic.

Interpretability:
- could help families understand the decision (although not as important as in diseases predictions).
- can also highlight and thus control biases (racial, sex, age).
- since the tool would probably be used in combination with human selection, it could help save time by highlighting the main factors for each decision

#### Conclusion:
- a model easily interpretable could be prefered (tree).
- or a highly accurate model (less interpretable) could also be used upfront (NN).


#### Future improvements

Imputations:
- Build more robust, generalisable imputations (eg. impute future test samples with missing gender based on all easily interpretable categories group_composition_label and group_composition_id)

- Automate NaNs imputation for future test samples

- Reconstruct some NaNs by training models to predict the feature

Feature engineering

Datasets handling
- I made the choice to keep the train and test sets split. This is to prevent gaining insight from the test set while doing pre-processing, analysis and while training the model. To prevent duplicate code, I thus placed most of the inner workings in the class Analysis, which makes the reading less fluent. To improve, see how to better combine the visual aspect of Jupyter Notebooks, while maintaining code standards like DRY philosophy.


# Plan

I. Load

1. Packages, classes, functions

2. Databases (no join)

II. Pre-process data 1:

1. Impute NaNs

2. Remove outliers

III. Analyze

1. Impact on target

2. Correlated features

IV. Pre-process data 2:

1. Transform categorical features

V. Build Model

1. Benchmakrs

2. Rush model using all data

3. Simple using principal components

4. More complex using specific features

5. Ensemble

VI. Predictions

1. Train and Predict

2. Hyperparameter tunning (split train set)

VII. Evaluate methods


# Classes, Functions & imports

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import log_loss
from sklearn import tree

#os.chdir('/Users/Pro/Desktop/Git_Contests/Predictions/Emergency_housing/')
from cobratools import Analysis

In [2]:
# Define the test scorer
def competition_scorer(y_true, y_pred):
    return log_loss(y_true, y_pred, sample_weight=10**y_true)

In [3]:
# General constants
PRINT_ON = True
PLOT_ON = True

## Load data

In [4]:
requests_train = pd.read_csv(filepath_or_buffer='data/requests_train.csv',
                            sep=',',
                            low_memory=False,
                            error_bad_lines=False)

requests_test = pd.read_csv(filepath_or_buffer='data/requests_test.csv',
                            sep=',',
                            low_memory=False,
                            error_bad_lines=False)

individuals_train = pd.read_csv(filepath_or_buffer='data/individuals_train.csv',
                                sep=',',
                                low_memory=False,
                                error_bad_lines=False)

individuals_test = pd.read_csv(filepath_or_buffer='data/individuals_test.csv',
                            sep=',',
                            low_memory=False,
                            error_bad_lines=False)

In [5]:
# Set index col as request id
requests_train.set_index('request_id', inplace=True)
requests_test.set_index('request_id', inplace=True)

## Join datasets

Since there are multiple requests by individuals and multiple individuals by request, the straightfoward approach would be to create columns for each individual' informations. This way, no information would be lost, but the curse of dimensionality is very near and the number of samples might be too low to extract useful information.

The chosen approach is rather to only keep the request dataset' columns, and feature engineer additional columns based on the individuals data, eg.:
- nb of past requests made by the same individual
- nb nights granted in past requests of the same individual(s)/group
- gender diversity of the group
- etc.

## Instanciate object analysis

In [6]:
# Instanciate analysis object with request data
analyze_train = Analysis(requests_train)
analyze_test = Analysis(requests_test)

# Define properties

# - target
target = 'granted_number_of_nights' 
analyze_test.target = target
analyze_train.target = target
# - shape
analyze_train.m = analyze_train.df.shape[0]
analyze_test.m = analyze_test.df.shape[0]
analyze_train.n = analyze_train.df.shape[1]
analyze_test.n = analyze_test.df.shape[1]

# Pre-process data 1

Methodology:

- Clean-up request dataset

- Feature engineer using indiv dataset

## Impute NaNs

Methodology:
- inspect NaNs on train set

- if pattern detected, apply modifications on train and test sets

Observations:
- reverse engineering: the system seems to derive group_composition_id from group_composition_label, and both are then necessary linked/redondant => drop group_composition_label

- there can be multiple individuals by request, and multiple requests by individual

Further:
- Impute 14 pregnancy NaNs from child_to_come (not useful for the current objective)

### NaNs train summary

In [8]:
# Get Na counts: by feature, by sample
na_ft_train, na_sp_train = analyze_train.get_na_counts()
na_ft_test, na_sp_test = analyze_test.get_na_counts()
if PRINT_ON:
    print('Train: NaNs count by feature\n\n', na_ft_train[na_ft_train!=0])
    #print('\n\nTest: NaNs count by feature\n\n', na_ft_test[na_ft_test!=0])

Train: NaNs count by feature

 child_to_come                145947
housing_situation_label       16748
long_term_housing_request    165556
town                         159959
victim_of_violence_type      234175
dtype: int64


### Impute child_to_come NaNs

Observation
- There are 145947 NaNs for child_to_come on the train set (in request)

- There are only 14 NaNs for pregnancy in train set (in individuals)

Hypotheses
- Hyp: child_to_come is True if any indiv of the group is pregnant

Conclusion
- => Impute child_to_come from the pregnancy in the group of indiv of the request

In [12]:
# This takes ±3min
if False:
    analyze_train.impute_child_to_come(df_indiv=individuals_train)
    analyze_test.impute_child_to_come(df_indiv=individuals_test)

### Impute housing_situation_label NaNs

Observation (train set)
- There are 16748 NaNs for child_to_come (in request)
- X NaNs for housing_situation_id (in request)

- X NaNs for housing_situation_label_2 (in individuals)
- X NaNs for housing_situation_id_2 (in individuals)

- 21 housing_situation_label (request)
- 22 housing_situation_id (request)

Hypotheses
- Hyp: link with housing_situation_id (request), housing_situation_label_2 and housing_situation_id_2 (individuals)

Conclusion
- => Impute housing_situation_label

In [171]:
analyze_train.get_col_uniques('housing_situation_id')

array([200,  10, 100, 170,  50, 120, 130, 160, 140,  80,  40, 220,  70,
        30, 150, 110,  90,  60, 190,  20, 180, 210])

#### Explore impact on target

Results: there seems not to have a significative direct correlation with target

In [ ]:
# Summary of target counts:
# ± half individuals granted >=1 night(s)
# which means < 50% of requests
# Rq: still nicely balanced dataset
target_counts = analyze_train.df[target].value_counts()

if PRINT_ON:
    print('Absolute\n', target_counts)
    print('\n\nPercentage\n', target_counts / n_samples * 100)

# Filter pregnants
mask_pregnant = analyze_train.df['pregnancy'] == True

# Count pregnants: 11k+ pregnant
n_pregnants = sum(mask_pregnant)

# Percentage ± 3%
pct_pregnants = n_pregnants/ n_samples * 100


# Correlation target & pregnancy: ± -.1% (significant? not)
analyze_train.df[[target, 'pregnancy']].corr()

# Summary
target_pregnancy_counts = analyze_train.df[target].groupby(analyze_train.df['pregnancy']).value_counts()

if PRINT_ON:
    # Pct pregnants by target
    print('\n\nN of nights granted for pregnants', target_pregnancy_counts[1] / n_pregnants)
    # Pct not pregnants by target
    print('\nN of nights granted for not pregnants', target_pregnancy_counts[0] / (n_samples - n_pregnants))

# => Repartitions are similar

#### Pattern for missing pregnancy?
Search if missing is treated as pregnant through correlated variables

In [ ]:
analyze_train.df['gender'].groupby(analyze_train.df['pregnancy']).value_counts()

In [ ]:
# Pattern with child_situation
analyze_train.df[target].groupby(analyze_train.df['pregnancy']).value_counts()

In [ ]:
# Requests granted (nb_nights > 0)
granted = analyze_train.df[target] > 0


## child_to_come

flag indicating whether the group is expecting a baby


What does it represent?

- hyp: it is the corresponding value of the individual "pregnancy"

Then?

- now that dataframes are joined, it is redondant

Conclusion

- use it to impute pregnancy NAs

- then drop the feature

# Outliers

## Gender

Only females are possibly pregnant, thus 30 males have made a mistake

In [ ]:
analyze_train.df['gender'].groupby(analyze_train.df['pregnancy']).value_counts()

In [ ]:
# Retrieve the individual ids, and correct for male -> pregnancy = 0

# Analyze data

## Analysis (df_train)
### General:
- Number of requests: 238191
- Number of individuals: 384133
- Number of features: 39

Requests are made for 1.6 pers on average.


### Correlations with granted_number_of_nights
- housing_situation_id: -0.458581. Strong negative impact. A high value must represent the good quality of the current housing situation.
- housing_situation_2_id: 0.283840. Strong positive impact. A higher value must conversely represent a degraded quality.

### Principal components
- housing_situation_2_label: with value "emergency accomodation". High probability to get 1 or two nights.

In [ ]:
Analyze_df_train = Analysis(df_train)
#Analyze_df_train.describe(investigation_level=3)
#Analyze.visualize()

In [ ]:
target = 'granted_number_of_nights'
feature = 'animal_presence'
mask = df_train[feature] == 't'
df_train[mask][[feature, target]]

In [ ]:
# Check for NA, inf
columns_selected = ['animal_presence']
df_train[columns_selected].isnull().sum()
df_train.isnull().sum()

## Analysis of features

feature: housing_situation_2_label
- A majority of requests with the label "emergency accomodation" obtains 1 or 2 nights.

In [ ]:
# Impact of feature on target
join_key = 'request_id'
target = 'granted_number_of_nights'
feature = 'housing_situation_2_label'
mask = df_train[feature] == 'emergency accomodation'

# Hist: drop duplicate requests (due to indiv data merged)
df_train[mask][[join_key, target]].drop_duplicates().hist()
plt.show()

## Transform categorical features
Prepare data to feed models

- booleans: replace by (1, 0)

- 2 < cats < 11: one-hot encoding

- No transform on features with more than 11 categories

In [ ]:
# Pre-process columns:
# booleans: 't', 't' => True, False
# Categorical with few classes => one-hot encoding
bools_train, failed_train = analyze_train.transform_categories(target=analyze_train.target)
bools_test, failed_test = analyze_test.transform_categories(target=analyze_test.target)

# Preprocess specific cat columns
analyze_train.convert_to_bool(col='group_type',
                            true_val='group',
                            false_val='individual')

# Preprocess specific cat columns
analyze_test.convert_to_bool(col='group_type',
                            true_val='group',
                            false_val='individual')

# Export data
analyze_train.export_data('data/data_train_preprocessed.csv')
analyze_test.export_data('data/data_test_preprocessed.csv')

# Predict

## Build models

### Benchmarks

In [ ]:
# Random uniform train/test
random_preds_train = np.random.uniform(size=(requests_train.shape[0], 4))
random_preds_test = np.random.uniform(size=(requests_test.shape[0], 4))

# Dumb (always pred 3)
dumb_preds_train = np.zeros((requests_train.shape[0], 4))
dumb_preds_test = np.zeros((requests_test.shape[0], 4))
# Set 10% pred everywhere (if not, log penalyzes hardly)
dumb_preds_train[:,:] = .01
dumb_preds_test[:,:] = .01
# Set 20% pred on class 3
dumb_preds_train[:,2] = .02
dumb_preds_test[:,2] = .02

### Univariate predictions

We observed a significant (negative) correlation of housing_situation_id with granted_number_of_nights, let's train a univariate model


In [ ]:
# Set model' parameters
clf = tree.DecisionTreeClassifier(
    criterion='gini',
    splitter='best',
    max_depth=2,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features=None,
    max_leaf_nodes=None,
    class_weight=None)

# Build train/test datasets with housing_situation_id only
X_train = np.array(requests_train['housing_situation_id']).reshape(-1, 1)
X_test = np.array(requests_test['housing_situation_id'].values).reshape(-1, 1)
Y_train = requests_train.granted_number_of_nights.values
Y_test = requests_test.granted_number_of_nights.values

# Transform categorical target into a one-hot vector
Y_train_onehot = to_onehot(Y_train)
Y_test_onehot = to_onehot(Y_test)

# Train the model
clf = clf.fit(X_train, Y_train_onehot)

# Yield train/test predictions
preds_train_tree_univar = clf.predict(X_train)
preds_test_tree_univar = clf.predict(X_test)

# Fill predictions to .2 elsewhere
preds_train_tree_univar[preds_train_tree_univar == 0] = .2
preds_test_tree_univar[preds_test_tree_univar == 0] = .2

# Evaluate train/test
score_train = competition_scorer(Y_train, preds_train_tree_univar)
score_test = competition_scorer(Y_test, preds_test_tree_univar)

# Display results
print(f'train score: {score_train:.2f}')
print(f'test score: {score_test:.2f}')

In [ ]:
probas = clf.predict_proba(X_train)
v0 = probas[0].max(1)
v1 = probas[1].max(1)
v2 = probas[2].max(1)
v3 = probas[3].max(1)

## Evaluate models

In [ ]:
y_true_test = requests_test.granted_number_of_nights.values

# Evaluate benchmarks
random_score_test = competition_scorer(y_true_test, random_preds_test)
dumb_score_test = competition_scorer(y_true_test, dumb_preds_test)

# Display results
print(f'test score random: {random_score_test:.2f}')
print(f'test score dumb: {dumb_score_test:.2f}')

### Train set

In [ ]:
y_true_train = requests_train.granted_number_of_nights.values

# Evaluate benchmarks
random_score_train = competition_scorer(y_true_train, random_preds_train)
dumb_score_train = competition_scorer(y_true_train, dumb_preds_train)

# Display results
print(f'train score random: {random_score_train:.2f}')
print(f'train score dumb: {dumb_score_train:.2f}')

### Test set

## Interpret models

In [ ]:
# Tree
fn = ['housing_situation_id']
cn = ['0', '1', '2', '3']

tree.plot_tree(clf,
               feature_names = fn, 
               class_names=cn,
               filled = True)

requests_train['housing_situation_id'].hist()